# Hyperparameter Tuning using HyperDrive

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import joblib

from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice

from azureml.widgets import RunDetails

from azureml.data.dataset_factory import TabularDatasetFactory

## Dataset

Get data. In the cell below, write code to access the data we will be using in this project. Remember that the dataset needs to be external.

In [ ]:
# Data Acquisition
ds = TabularDatasetFactory.from_delimited_files("https://raw.githubusercontent.com/eparamasari/ML_Engineer_ND_Capstone/main/data/heart_failure_clinical_records_dataset.csv")

In [ ]:
ws = Workspace.from_config()
experiment_name = 'ClassificationHearthFailure-HyperDrive'

experiment=Experiment(ws, experiment_name)

In [ ]:
# Checking an existing compute cluster or starting one
compute_name = "HD-Compute"
try:
    compute_target = ComputeTarget(ws, compute_name)
    print(compute_name+ " already exists.")
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_DS3_V2", min_nodes=1, max_nodes=5)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)
compute_target.wait_for_completion(show_output=True)

print(compute_target.get_status().serialize())

## Hyperdrive Configuration

Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# Creating an early termination policy with Random parameter sampling
early_termination_policy = BanditPolicy(evaluation_interval=1, 
                                        slack_factor=0.2, 
                                        delay_evaluation=5)

# Creating the different parameters that will be used during training
param_sampling = RandomParameterSampling(
     {
        '--n_estimators': choice(20,40), 
        '--min_samples_split': choice(2,4,6)
     })

# Creating an environment, script run config and hyperdrive config
from azureml.core import ScriptRunConfig
from azureml.core.environment import Environment

myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")

src = ScriptRunConfig(compute_target=compute_target,
                    source_directory='./',
                    script='train.py',
                    environment=myenv)

hyperdrive_run_config = HyperDriveConfig(run_config=src,
                                         policy=early_termination_policy,
                                         hyperparameter_sampling=param_sampling,
                                         primary_metric_name="Accuracy",
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=50,
                                         max_concurrent_runs=4)

In [ ]:
# Submitting the experiment
hyperdrive_run = experiment.submit(hyperdrive_run_config, show_output=True)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()

In [ ]:
hyperdrive_run.wait_for_completion(show_output=True)

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
best_hd_run_metrics = best_hyperdrive_run.get_metrics()

print('Best Run Id: ', best_hyperdrive_run.id)
print('\n Best Run Metrics: ', best_hd_run_metrics)

In [ ]:
print('\n Accuracy: ', best_hd_run_metrics['Accuracy'])
print('\n N Estimators: ', best_hd_run_metrics['The number of trees in the forest:'])
print('\n Min Samples Split: ', best_hd_run_metrics['The minimum number of samples required to split an internal node:'])

In [ ]:
# Saving the best model
os.makedirs("./outputs", exist_ok=True)
joblib.dump(value=best_hyperdrive_run.id,filename='outputs/best_hyperdrive_run_model.joblib')
print("Model has been successfully saved!")

In [ ]:
# Registering the best model with metrics information
model = best_hyperdrive_run.register_model(model_name='heart_failure_hyperdrive', model_path='outputs/', 
                    properties={'Accuracy': best_hd_run_metrics['Accuracy'],
                                'N Estimators': best_hd_run_metrics['The number of trees in the forest:'],
                                'Min Samples Split': best_hd_run_metrics['The minimum number of samples required to split an internal node:']})

In [ ]:
# Listing registered models to verify that the model has been saved
for model in model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')